## Imports

In [1]:
import numpy as np
import cv2
import mediapipe as mp
import os
import csv
import matplotlib.pyplot as plt 
from utils import *

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

hyperparameters of the video paths for downstream testing

In [2]:
video_1 = "./Video1.mp4"
video_2 = "./Video2.mp4"
video_3 = "./Video3.mp4"

## Testing the cv2 window

In [9]:
cap = cv2.VideoCapture(video_1)
 
if not cap.isOpened():
    print("Error: Unable to open video file.")
    exit()
 
while True: 
    ret, frame = cap.read()
 
    if not ret:
        print("Error: Unable to read frame.")
        break
    cv2.imshow("Video", frame)
 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

## Joint Detection

In [13]:
cap = cv2.VideoCapture(video_1) 

with mp_pose.Pose(min_detection_confidence=0.2, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 
        results = pose.process(image) 
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      
        try:
            landmarks = results.pose_landmarks.landmark 
        except:
            pass
         
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
        )               
        
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [23]:
print("We were able to detect {} number of joints or landmarks.".format(len(landmarks)))
print("The right shoulder was visible with {} % visibility".format(round(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].visibility*100, 2)))
print("The right elbow was visible with {} % visibility".format(round(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].visibility*100, 2)))
print("The co-ordinates of the right elbow were x = {} and y = {} and z = {}".format(round(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 3), round(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, 3), round(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z, 3)))

We were able to detect 33 number of joints or landmarks.
The right shoulder was visible with 99.99 % visibility
The right elbow was visible with 98.78 % visibility
The co-ordinates of the right elbow were x = 0.209 and y = 0.243 and z = -0.11


This function is used to find angle between any three joint points at the central joint

In [7]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b) 
    c = np.array(c) 
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    if angle >180.0:
        angle = 360-angle
    return angle 

In [8]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [9]:
shoulder, elbow, wrist

([0.5303999781608582, 0.3968207836151123],
 [0.5283608436584473, 0.49190232157707214],
 [0.5301544070243835, 0.574292004108429])

In [10]:
calculate_angle(shoulder, elbow, wrist)

177.5243230982769

In [19]:
cap = cv2.VideoCapture(video_1)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image) 
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        try:
            landmarks = results.pose_landmarks.landmark 
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
             
            l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
            height, width, channels = image.shape
         
            cv2.putText(
                image, str(l_angle), 
                tuple(np.multiply(left_elbow, [width, height]).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            )    

            cv2.putText(
                image, str(r_angle), 
                tuple(np.multiply(r_elbow, [width, height]).astype(int)), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            )        
        except:
            pass
         
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
        )               
        
        cv2.imshow('Mediapipe Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

## Detection of illegal bowling

In [14]:
bowler = 'right arm'
state = 'OK'
cap = cv2.VideoCapture(video_2)   
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read() 
        if ret:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False 
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
            try:
                landmarks = results.pose_landmarks.landmark 

                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]

                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                l_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                r_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                right_leg = calculate_angle(right_hip, right_knee, right_ankle)
                left_leg = calculate_angle(left_hip, left_knee, left_ankle)
                height, width, channels = image.shape

                cv2.putText(image, str(l_angle), 
                            tuple(np.multiply(left_elbow, [width, height]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )

                cv2.putText(image, str(r_angle), 
                            tuple(np.multiply(r_elbow, [width, height]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )      

                cv2.putText(image, str(right_leg), 
                            tuple(np.multiply(right_knee, [width, height]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )   
                
                cv2.putText(image, str(left_leg), 
                            tuple(np.multiply(left_knee, [width, height]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )        
                
                ## Main Logic Of the Ilegal Action Detector ##
                if (bowler == 'right arm'):
                    if (right_leg <160) and (left_leg >170):
                        if (r_angle<150):
                            state = 'Not Allowed'
                        else:
                            state = 'OK'
                elif (bowler == 'left arm'):
                    if (left_leg <160) and (right_leg >170):
                        if (l_angle<150):
                            state = 'Not Allowed'
                        else:
                            state = 'OK'
            except:
                pass
            
            cv2.rectangle(image, (0, 0), (325, 73), (245, 117, 16), -1)  
            cv2.rectangle(image, (430, 0), (250, 73), (245, 117, 16), -1)
            
            cv2.putText(image, 'STATE', (25,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, state, 
                        (20,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

In [3]:
Analyzer = BowlingActionAnalyser(video=video_1, bowler='right arm', position=0)
Analyzer.analyze()